In [7]:
!pip install spacy #installation SpaCy pour l'analyse de texte (tokenisation, lemmatisation, reconnaissance d'entités nommées...)

!python -m spacy download fr_core_news_sm # Téléchargement de  modèle linguistique français de petite taille pour SpaCy.
# Ce modèle est nécessaire pour le traitement de texte en français-->offre des fonctionnalités comme l'analyse syntaxique, la lemmatisation et la reconnaissance d'entités nommées.

!pip install unidecode #pour supprimer l'accent
!pip install pyspellchecker #pour correction des orthographes

#c'est l'installation des biblio



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 89.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.3 MB/s eta 0:00:00


In [8]:
import requests  # faire des requêtes Http

from bs4 import BeautifulSoup  # extraire et analyser du contenu HTML

import spacy  #le traitement du langage naturel (NLP)

import string  # pour supprimer ponctuation

from nltk.corpus import stopwords, wordnet as wn  # pour suppression des mots vides et gérer les synonymes les synonymes

import nltk

from unidecode import unidecode  # Pour supprimer les accents des caractères

In [9]:
# Télécharge les ressources nécessaires de NLTK : les mots vides (stopwords), la base de synonymes (wordnet) et les données supplémentaires pour WordNet (omw-1.4)

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [10]:
# Chargement du modèle français de spaCy
nlp = spacy.load("fr_core_news_sm")

In [11]:
# Chargement des mots vides
stop_words = set(stopwords.words('french'))

In [12]:
# Fonction pour extraire le contenu d'une page
#Fonction pour extraire le contenu structuré d'une page : envoie une requête à l'URL, analyse le HTML pour trouver le contenu principal, extrait les sections de titres (h2, h3) et leur contenu (paragraphes, listes),
# puis organise ces informations en une liste de dictionnaires contenant l'URL, le titre de la section et le texte de contenu , renvoie un message d'erreur si la requête échoue.

def extract_content(url):
    try:
        response = requests.get(url, timeout=10)  # Limite de 10 secondes
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            main_content = soup.find('div', {'id': 'mw-content-text'})
            documents = []
            if main_content:
                sections = main_content.find_all(['h2', 'h3', 'p', 'ul', 'ol'])
                current_section_title = None
                for section in sections:
                    if section.name in ['h2', 'h3']:
                        current_section_title = section.text.strip()
                    elif section.name in ['p', 'ul', 'ol'] and current_section_title:
                        current_section_content = section.text.strip()
                        documents.append({
                            'url': url,
                            'title': current_section_title,
                            'content': current_section_content
                        })
            return documents
        else:
            print(f"Erreur lors de la récupération de la page {url} : {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération de l'URL {url}: {e}")
    return []


In [13]:

#Une fonction défini pour corriger automatiquement les erreurs typographiques courantes dans des textes en français(des erreurs basiques pas complexes s'il existe )
# (dans mon cas je vais utiliser lemmatisation pour éviter ces problèmes .

from spellchecker import SpellChecker

# Initialiser SpellChecker pour le français
spell = SpellChecker(language='fr')

# Fonction pour corriger un mot en français
def correct_word(word):
    corrected_word = spell.correction(word)
    return corrected_word

# Exemple d'utilisation
word = "bonj"
print(f"Correction pour '{word}': {correct_word(word)}")

Correction pour 'bonj': bon



**Maintenat pré traitement de texte en une seule fonction:**


1.   **Suppression de la ponctuation et des chiffres**(car les chiffres ici ont un poids faible)
2.   **Analyse et tokenisation avec SpaCy** : segmente le texte en tokens /identifie également les entités nommées, comme les noms de personnes, lieux et dates, qui sont remplacées par des balises spécifiques (<PERSON>, <LOCATION>, <DATE>)/


3.   **Stop Words et mots indésirables**
4. **Lemmatisation:**L'objectif est de traiter les variantes linguistiques d'un même mot comme une seule entité





5. **la plus imporante étape:**

     **1/Prise du mot et recherche de son synonyme :** Pour chaque mot, on effectue une recherche de synonymes dans un dictionnaire de synonymes. On prend systématiquement le premier synonyme trouvé, et ce, quel que soit le type grammatical du mot d'origine (qu'il soit verbe, nom, adjectif, ou adverbe).

      **2/Uniformisation avec un synonyme verbal :** Tous les mots sont normalisés en prenant un synonyme ayant la forme d'un verbe (premier synonyme trouvé dans la catégorie verbale). Cette uniformisation se fait de manière systématique pour tous les mots ayant un sens similaire.

      **3/Obtention d'une même forme pour des mots différents mais synonymes :** Un autre mot ayant un sens similaire subira le même processus, ce qui signifie que plusieurs mots aux sens proches seront remplacés par le même mot sous une forme verbale unique. **Par exemple, "rapide" et "vite" pourraient tous deux être remplacés par le verbe "accélérer".-->Cette sélection permet de regrouper tous les mots synonymes sous une forme unique.**
      **-----> : Cette uniformisation facilite ensuite une indexation plus précise basée sur le sens. Si plusieurs mots synonymes apparaissent dans un document, ils seront représentés par un même mot unique**


In [14]:
from nltk.corpus import wordnet as wn

# Fonction pour obtenir les synonymes en fonction du mot et de la catégorie grammaticale
def get_synonyms(word, pos):
    synonyms = set()

    # Utilise la catégorie grammaticale pour filtrer les synonymes
    wordnet_pos = None
    if pos == "NOUN":
        wordnet_pos = wn.NOUN
    elif pos == "VERB":
        wordnet_pos = wn.VERB
    elif pos == "ADJ":
        wordnet_pos = wn.ADJ
    elif pos == "ADV":
        wordnet_pos = wn.ADV

    # Recherche des synonymes dans WordNet avec la catégorie grammaticale spécifiée
    if wordnet_pos:
        for syn in wn.synsets(word, pos=wordnet_pos, lang='fra'):  # Recherche dans WordNet pour le français
            for lemma in syn.lemmas('fra'):
                synonyms.add(lemma.name())  # Ajoute le nom du synonyme dans l'ensemble des résultats

    return synonyms



In [15]:
# Fonction pour prétraiter les documents
def preprocess_documents(documents):
    processed_documents = []
    undesirable_words = ["l'", "d'", "s'", "s’", "d’", "l’","c’"] #pour éviter la confusion entre les formes de virgules car on trouve "d'" et "d’".


    for doc in documents:
        content_without_punctuation = ''.join(char for char in doc['content'] if char not in string.punctuation)
        content_normalized = ''.join(char if not char.isdigit() else ' ' for char in content_without_punctuation).lower() #le poids des nombre est faible on l'élimigne dés le débit et on convertit tout le texte en miniscule pour etre traité.


        doc_spacy = nlp(content_normalized)
        tokens = []

        # traitement des entités nommées : remplace les noms de personnes, lieux et dates par des balises spécifiques
        for ent in doc_spacy.ents:
            if ent.label_ == "PER":
                tokens.append("<PERSON>")
            elif ent.label_ == "LOC":
                tokens.append("<LOCATION>")
            elif ent.label_ == "DATE":
                tokens.append("<DATE>")
          # Tokenisation et normalisation des mots

        for token in doc_spacy:
          # filtre les stop words et un mot spécifique 'leurs' car 'leurs' n'a pas été supprimé avec les stops words

            if (token.text.lower() not in stop_words and token.text.lower() != 'leurs'):
                lemma = token.lemma_.lower().strip() #pour supprimer les espaces

                 # exclut les mots indésirables définis précédemment

                if lemma not in undesirable_words:
                    target_pos = "NOUN" if token.pos_ in ["VERB", "ADJ", "ADV"] else token.pos_
                    synonyms = get_synonyms(lemma,target_pos) # récupère les synonymes du mot

                    if synonyms:
                        standardized = next(iter(synonyms)) # prend le premier synonyme toujours
                        correct_word(standardized) # correction orthographique
                        standardized = unidecode(standardized)  # suppression des accents
                        tokens.append(standardized)
                    else:
                        if lemma:
                            correct_word(lemma)
                            lemma= unidecode(lemma)
                            tokens.append(lemma)
       # ajout de document traité (avec URL, titre, et tokens) à la liste des documents traités


        processed_documents.append({
            'url': doc['url'],
            'title': doc['title'],
            'tokens': tokens
        })

    return processed_documents

remarque : chaque bloc dans le site web est considéré comme un seul unique document d'ou dans chaque site web on trouve plusieurs documenent.**

In [16]:
# Fonction pour afficher les documents prétraités
def display_processed_documents(processed_documents):
    print("\nDocuments prétraités :\n")
    for proc_doc in processed_documents:
        print(f"Document : {proc_doc['title']} (URL: {proc_doc['url']})")
        print(f"Tokens prétraités : {proc_doc['tokens']}\n")

In [17]:
#  maintenat on applique les fonctions précédentes Liste des URL à traiter
urls = [
     "https://fr.wikipedia.org/wiki/Wikipédia:Accueil_principal",
    "https://fr.wikipedia.org/wiki/France",
    "https://fr.wikipedia.org/wiki/Technologie",
    "https://fr.wikipedia.org/wiki/Économie",
    "https://fr.wikipedia.org/wiki/Éducation"
]

# exécution des fonctions
all_documents = []
for url in urls:
    all_documents.extend(extract_content(url))

processed_documents = preprocess_documents(all_documents)
display_processed_documents(processed_documents)


Documents prétraités :

Document : Wikipédia (URL: https://fr.wikipedia.org/wiki/Wikipédia:Accueil_principal)
Tokens prétraités : ['<LOCATION>', 'lencyclopedie', 'libre', 'effet', 'ameliorer']

Document : Wikipédia (URL: https://fr.wikipedia.org/wiki/Wikipédia:Accueil_principal)
Tokens prétraités : ['armee', "zone_d'habitation", 'comment', 'contribuer', 'portail', 'thematique', 'dogme', 'pere', 'Sommaire', 'laide', 'place', 'interrogatif']

Document : Article labellisé du jour (URL: https://fr.wikipedia.org/wiki/Wikipédia:Accueil_principal)
Tokens prétraités : ['<PERSON>', '<PERSON>', '<PERSON>', 'yuri', 'ice', 'yu-ri', 'ic', 'parfois', 'styliser', 'yuri', 'ic', 'armee', 'televiser', 'danimation', 'nippon', 'patinage', 'artistique', 'rendement', 'studio', 'mappa', 'realiser', 'sayo', 'Yamamoto', 'autorisation', 'dun', 'figure', 'pilote', 'ecrire', 'mitsuro', 'kubo', 'musique', 'composer', 'taro', 'umebayashi', 'taku', 'matsushiber', 'bien_que', 'kenji', 'miyamoto', 'skateuse', 'plusie

calcul de Tf

In [18]:
from collections import Counter

# fonction pour calculer TF (nombre d'occurrences d'un mot dans un document spécifique) pour chaque document
def calculate_tf_occurrences(processed_documents):
    tf_documents = []
    for doc in processed_documents:
        # Compter le nombre d'occurrences des tokens dans le document
        token_counts = Counter(doc['tokens'])

        tf_documents.append({
            'url': doc['url'],
            'title': doc['title'],
            'tf': token_counts  # stocker les occurrences directement
        })

    return tf_documents

tf_documents = calculate_tf_occurrences(processed_documents)

affichage de tf

In [19]:
# fonction pour afficher uniquement les TF en nombre d'occurrences
def display_tf(tf_documents):
    print("\nTerm Frequency (Nombre d'occurrences) pour chaque document :\n")
    for doc in tf_documents:
        print(f"Document : {doc['title']} (URL: {doc['url']})")
        print("TF :")
        for token, tf_value in doc['tf'].items():
            print(f"   {token}: {tf_value}")
        print("\n")

display_tf(tf_documents)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
   etre: 5
   frapper: 1
   percuteur: 2
   egalement: 1
   intervalle: 1
   detacher: 2
   eclat: 1
   ce: 1
   action_en_justice: 1
   rendement: 1
   corne: 1
   coupant: 1
   quatrieme_dimension: 1
   tailler: 2
   leclat: 2
   avoir: 1
   femme_de_menage: 1
   deux: 1
   effet: 3
   grace: 2
   rondelle: 1
   variete: 1
   racloir: 1
   premiere_vitesse: 3
   chasseurscueilleur: 1
   executer: 1
   differents: 1
   preparation: 1
   dont: 1
   decoupe: 1
   noyau: 2
   repartition: 1
   perle: 1
   acceder: 1
   osseux: 1
   coiffure: 1
   ebene: 2
   louverture: 1
   couille: 1
   depouillemer: 1
   carcass: 1
   faune: 2
   recuperer: 1
   pulpe: 1
   entourage: 1
   fabriquer: 1
   dautre: 1
   materiel: 1
   tendre: 1
   los: 1
   relativement: 1
   moment: 1
   elabore: 1
   techniquement: 1
   impliquer: 1
   commandement: 1
   dun: 2
   ensemble: 1
   important: 1
   parametre: 1
   faculte: 2
   

affichage de df

In [20]:
from collections import Counter

# fonction pour calculer la fréquence des documents (DF)
def calculate_df(processed_documents):
    df_counter = Counter()

    for doc in processed_documents:
        # obtenir les mots uniques de chaque document
        unique_tokens = set(doc['tokens'])

        # mettre à jour le compteur DF pour chaque mot unique
        df_counter.update(unique_tokens)

    return df_counter

df = calculate_df(processed_documents)

In [21]:
# fonction pour afficher la fréquence des documents (DF)
def display_df(df):
    print("\nDocument Frequency (DF) pour chaque mot :\n")
    for token, df_value in df.items():
        print(f"   {token}: {df_value}")

display_df(df)



Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
   celler: 1
   gouvernemental: 3
   separer: 3
   divorce: 2
   imminentee: 1
   syndicat: 5
   pas_de_vis: 3
   boulangiste: 1
   decoration: 1
   Maroc: 5
   lexterieur: 2
   lecole: 8
   panama: 2
   laique: 3
   qua: 3
   lindochine: 1
   lalgerie: 1
   chaotique: 1
   obligatoire: 11
   dreyfu: 4
   sajouter: 1
   protectorat: 1
   Hyderabad: 2
   madagascar: 1
   accorder: 4
   Tunisie: 4
   outrage: 1
   lafriqu: 2
   mosquee: 6
   la_soupe_au_canard: 5
   daout: 1
   assurance: 1
   fiche: 12
   conclure: 2
   larmee: 2
   compensation: 1
   tripleententee: 1
   faveur: 4
   lune: 3
   victime: 2
   convent: 3
   suffire: 2
   rivage: 1
   gauche: 5
   lalsacelorraine: 1
   la_Reconstruction: 2
   allemand: 2
   devastation: 2
   remilitarisation: 1
   eviter: 1
   obtenant: 1
   cellesci: 2
   retrouver: 3
   nazi: 2
   lespoir: 1
   declarer: 5
   larriver: 2
   republiquee: 1
   juste: 1
   labori

In [31]:
# fonction pour calculer le nombre total de mots dans tous les documents
def calculate_total_word_count(processed_documents):
    total_word_count = sum(len(doc['tokens']) for doc in processed_documents)
    return total_word_count

total_word_count = calculate_total_word_count(processed_documents)

print("Nombre total de mots dans tous les documents :", total_word_count)

Nombre total de mots dans tous les documents : 30989



**creation d'un index inversé**

{
    "mot1": {
        "Document1": fréquence1,
        "Document2": fréquence2
    },
    "mot2": {
        "Document1": fréquence3,
        "Document3": fréquence4
    },
    ...
}

In [23]:
# fonction pour créer un index inversé avec les mots comme clés principaux
def create_word_index(tf_documents):
    word_index = {}

    # parcourir chaque document et chaque mot avec sa fréquence
    for doc in tf_documents:
        title = doc['title']
        for word, count in doc['tf'].items():
            # si le mot n'est pas dans l'index, l'ajouter avec un dictionnaire vide
            if word not in word_index:
                word_index[word] = {}
            word_index[word][title] = count

    return word_index

word_index = create_word_index(tf_documents)

print("Index des mots :")
print(word_index)


Index des mots :
{'<LOCATION>': {'Wikipédia': 1, 'Article labellisé du jour': 1, 'Actualités': 1, 'Éphéméride du 10 novembre': 8, 'Toponymie': 1, 'Localisation, frontières et superficie': 2, 'Géologie, topographie et hydrographie': 1, 'Climat': 2, 'Paysages et environnement': 1, "Préservation de l'environnement": 1, 'Répartition spatiale des hommes et des activités': 4, 'Axes de communication et de transports': 2, 'Histoire': 2, 'Préhistoire, protohistoire et Antiquité': 4, 'Naissance, crises et transformations du royaume de France au Moyen Âge': 4, 'De la Renaissance à la monarchie absolue (XVIe au XVIIIe\xa0siècle)': 5, 'Révolutions, républiques, monarchies et empires (1789-1914)': 5, 'La France dans les deux guerres mondiales (1914-1945)': 2, 'La Libération, gaullisme et crise politique (1945-1968)': 3, 'Depuis Mai 68': 1, 'Politique et administration': 3, 'Organisation des pouvoirs': 2, 'Découpage territorial et décentralisation': 1, "France d'outre-mer": 1, 'Tendances politiques, 

**Filtrage des mots avec faibles frèquences en utilisant dess seuils de tf et de df  --> chaque document contient uniquement les mots dépassant les seuils de TF et DF.**
{
    "mot": {
        "titre_document": fréquence du mot dans le_document,
        "titre_document_suivant": fréquence du mot dans le document_suivant,
        ...
    },
    ...
}


In [24]:
from collections import Counter

def create_mot_index(processed_documents):
    mot_index = {}

    # parcourt de chaque document et compter les occurrences de chaque mot
    for doc in processed_documents:
        title = doc['title']  # nom du document pour l'indexation
        token_counts = Counter(doc['tokens'])  # compter les occurrences des tokens dans le document

        # Construction l'index avec les mots comme clés principaux
        for token, count in token_counts.items():
            if token not in mot_index:
                mot_index[token] = {}
            mot_index[token][title] = count  # Associer le document et la fréquence du mot

    return mot_index

# Génération de mot_index à partir des documents traités
mot_index = create_mot_index(processed_documents)


In [25]:
# fonction pour filtrer les mots en fonction de TF et DF
def filtrer_mots_df_tf(mot_index, seuil_tf=2, seuil_df=2):

    index_filtré = {}

    for mot, docs in mot_index.items():
        # calculer DF : nombre de documents dans lesquels le mot apparaît
        df = len(docs)

        # filtrer les mots selon le seuil de DF
        if df >= seuil_df:
            # Filtrer les occurrences du mot dans chaque document selon le seuil de TF
            docs_filtrés = {doc: tf for doc, tf in docs.items() if tf >= seuil_tf}

            # Si le mot a encore des occurrences après filtrage TF, on le conserve
            if docs_filtrés:
                index_filtré[mot] = docs_filtrés

    return index_filtré

# la fonction pour filtrer l'index des mots selon les seuils souhaités
index_filtré = filtrer_mots_df_tf(mot_index, seuil_tf=2, seuil_df=2)

# Affichage le dictionnaire filtré
print("Dictionnaire filtré selon les seuils TF et DF :")
print(index_filtré)


Dictionnaire filtré selon les seuils TF et DF :
{'<LOCATION>': {'Éphéméride du 10 novembre': 8, 'Localisation, frontières et superficie': 2, 'Climat': 2, 'Répartition spatiale des hommes et des activités': 4, 'Axes de communication et de transports': 2, 'Histoire': 2, 'Préhistoire, protohistoire et Antiquité': 4, 'Naissance, crises et transformations du royaume de France au Moyen Âge': 4, 'De la Renaissance à la monarchie absolue (XVIe au XVIIIe\xa0siècle)': 5, 'Révolutions, républiques, monarchies et empires (1789-1914)': 5, 'La France dans les deux guerres mondiales (1914-1945)': 2, 'La Libération, gaullisme et crise politique (1945-1968)': 3, 'Politique et administration': 3, 'Organisation des pouvoirs': 2, 'Tendances politiques, partis et élections': 4, 'Finances publiques': 3, 'Défense': 7, 'Appartenance à des organisations internationales': 3, 'Immigration, population étrangère et minorités visibles': 4, 'Famille, sexualité et égalité des sexes': 2, 'Langues': 2, 'Santé et protec

**{
    "mot": {
        "titre_document": fréquence du mot dans le document,
        "titre_document_suivant": fréquence du mot dans le document suivant,
        ...
    },
    ...
}**:

In [35]:
# fonction pour afficher l'index inversé filtré sous forme détaillée
def afficher_documents_filtrés_détaillé(index_inversé_filtré):
    print("{")
    for mot, docs in index_inversé_filtré.items():
        print(f'  "{mot}": {{')
        for i, (titre_document, fréquence) in enumerate(docs.items()):
            # Ajouter une virgule après chaque document sauf le dernier
            end = "," if i < len(docs) - 1 else ""
            print(f'    "{titre_document}": {fréquence}{end}')
        print("  },")
    print("}")

# Exemple d'appel de la fonction
afficher_documents_filtrés_détaillé(index_filtré)





{
  "<LOCATION>": {
    "Éphéméride du 10 novembre": 8,
    "Localisation, frontières et superficie": 2,
    "Climat": 2,
    "Répartition spatiale des hommes et des activités": 4,
    "Axes de communication et de transports": 2,
    "Histoire": 2,
    "Préhistoire, protohistoire et Antiquité": 4,
    "Naissance, crises et transformations du royaume de France au Moyen Âge": 4,
    "De la Renaissance à la monarchie absolue (XVIe au XVIIIe siècle)": 5,
    "Révolutions, républiques, monarchies et empires (1789-1914)": 5,
    "La France dans les deux guerres mondiales (1914-1945)": 2,
    "La Libération, gaullisme et crise politique (1945-1968)": 3,
    "Politique et administration": 3,
    "Organisation des pouvoirs": 2,
    "Tendances politiques, partis et élections": 4,
    "Finances publiques": 3,
    "Défense": 7,
    "Appartenance à des organisations internationales": 3,
    "Immigration, population étrangère et minorités visibles": 4,
    "Famille, sexualité et égalité des sexes": 

**{
    "titre_document": {
        "url": "url_du_document",
        "positions": {
            "mot1": [position1, position2, ...],
            "mot2": [position1, position2, ...],
            ...
        }
    },
    "titre_document_suivant": {
        "url": "url_du_document_suivant",
        "positions": {
            "mot3": [position1, position2, ...],
            "mot4": [position1, position2, ...],
            ...
        }
    },
    ...
}**

In [27]:
# fonction pour créer un dictionnaire de mots avec leurs positions dans le document
def creer_dictionnaire_positions(processed_documents):
    dictionnaire_positions = {}

    # parcourir chaque document traité
    for doc in processed_documents:
        title = doc['title']
        tokens = doc['tokens']

        # dictionnaire pour stocker les positions des mots pour le document courant
        positions_par_mot = {}

        # parcourir chaque token avec son index dans le document
        for index, token in enumerate(tokens):
            if token not in positions_par_mot:
                positions_par_mot[token] = []
            # Ajouter la position du mot dans la liste des positions
            positions_par_mot[token].append(index)

        # ajouter le dictionnaire de positions du document dans le dictionnaire principal
        dictionnaire_positions[title] = {
            'url': doc['url'],
            'positions': positions_par_mot
        }

    return dictionnaire_positions

# Créer le dictionnaire des positions à partir des documents prétraités
dictionnaire_positions = creer_dictionnaire_positions(processed_documents)

# afficher le dictionnaire des positions
print("Dictionnaire des positions des mots :")
for title, data in dictionnaire_positions.items():
    print(f"\nDocument : {title} (URL: {data['url']})")
    print("Positions des mots :")
    for token, positions in data['positions'].items():
        print(f"   {token}: {positions}")


Dictionnaire des positions des mots :

Document : Wikipédia (URL: https://fr.wikipedia.org/wiki/Wikipédia:Accueil_principal)
Positions des mots :
   armee: [0]
   zone_d'habitation: [1]
   comment: [2]
   contribuer: [3]
   portail: [4]
   thematique: [5]
   dogme: [6]
   pere: [7]
   Sommaire: [8]
   laide: [9]
   place: [10]
   interrogatif: [11]

Document : Article labellisé du jour (URL: https://fr.wikipedia.org/wiki/Wikipédia:Accueil_principal)
Positions des mots :
   contenir: [0, 3]
   caractere: [1]
   mot: [2]
   extrait: [4]
   programm: [5]

Document : Actualités (URL: https://fr.wikipedia.org/wiki/Wikipédia:Accueil_principal)
Positions des mots :
   Nicolas: [0]
   wikinews: [1]
   modifier: [2]

Document : Le saviez-vous ? (URL: https://fr.wikipedia.org/wiki/Wikipédia:Accueil_principal)
Positions des mots :
   archiver: [0]
   propos: [1]
   programm: [2]
   modifier: [3]

Document : Présentation (URL: https://fr.wikipedia.org/wiki/Wikipédia:Accueil_principal)
Positions de

**{mot: {nom_document: [positions]}}**

In [36]:
# fonction pour créer un dictionnaire {mot: {nom_document: [positions]}}
def creer_dictionnaire_positions_par_mot(processed_documents):
    dictionnaire_positions = {}

    # parcourir chaque document traité
    for doc in processed_documents:
        title = doc['title']
        tokens = doc['tokens']

        # parcourir chaque token avec son index dans le document
        for index, token in enumerate(tokens):
            # Si le mot (token) n'est pas encore dans le dictionnaire, on l'ajoute
            if token not in dictionnaire_positions:
                dictionnaire_positions[token] = {}

            # Si le document courant n'est pas encore associé à ce mot, on l'ajoute
            if title not in dictionnaire_positions[token]:
                dictionnaire_positions[token][title] = []

            # ajout de la position du mot dans la liste des positions pour ce document
            dictionnaire_positions[token][title].append(index)

    return dictionnaire_positions

# création de dictionnaire des positions par mot à partir des documents prétraités
dictionnaire_positions = creer_dictionnaire_positions_par_mot(processed_documents)

# affichage de dictionnaire des positions par mot
print("Dictionnaire des positions par mot :")
for mot, documents in dictionnaire_positions.items():
    print(f"\nMot : {mot}")
    for doc, positions in documents.items():
        print(f"  Document : {doc}")
        print(f"    Positions : {positions}")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  Document : Paléolithique
    Positions : [147]

Mot : depens
  Document : Paléolithique
    Positions : [148]

Mot : nucleus
  Document : Paléolithique
    Positions : [150]

Mot : generalise
  Document : Paléolithique
    Positions : [158]

Mot : burin
  Document : Paléolithique
    Positions : [163]

Mot : grattoir
  Document : Paléolithique
    Positions : [164]

Mot : retouche
  Document : Paléolithique
    Positions : [167]

Mot : middle
  Document : Paléolithique
    Positions : [171]
  Document : Références
    Positions : [376]

Mot : ston
  Document : Paléolithique
    Positions : [172]

Mot : retoucher
  Document : Paléolithique
    Positions : [177]

Mot : alcove
  Document : Paléolithique
    Positions : [183]

Mot : exact
  Document : Paléolithique
    Positions : [12]

Mot : etrangere
  Document : Paléolithique
    Positions : [14]

Mot : do
  Document : Paléolithique
    Positions : [16]

Mot


**un index inversé enrichi {mot: {nom_document: {frequence: positions}}}**

In [47]:
# fonction pour créer un index inversé enrichi {mot: {nom_document: {frequence: positions}}}
def creer_index_inverse_enrichi(processed_documents):
    index_inverse_enrichi = {}

    # Parcourt
    for doc in processed_documents:
        title = doc['title']
        tokens = doc['tokens']

        positions_par_mot = {}

        for index, token in enumerate(tokens):
            if token not in positions_par_mot:
                positions_par_mot[token] = []
            positions_par_mot[token].append(index)

        # Ajout les informations au dictionnaire principal avec la fréquence et les positions
        for token, positions in positions_par_mot.items():
            if token not in index_inverse_enrichi:
                index_inverse_enrichi[token] = {}

            # Ajout de document et ses informations de fréquence et positions pour le mot
            index_inverse_enrichi[token][title] = {
                'tf': len(positions),  # fréquence est la longueur de la liste
                'positions': positions  # Liste des positions
            }

    return index_inverse_enrichi

# Création de l'index inverse enrichi à partir des documents prétraités
index_inverse_enrichi = creer_index_inverse_enrichi(processed_documents)

# Affichage de l'index inverse enrichi
print("Index inversé enrichi des mots (créé par la fonction creer_index_inverse_enrichi) :")
for mot, documents in index_inverse_enrichi.items():
    print(f"\nMot : {mot}")
    for doc, info in documents.items():
        frequence = info['tf']
        positions = info['positions']
        print(f"  Document : {doc}")
        print(f"    Fréquence : {frequence}")
        print(f"    Positions : {positions}")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  Document : Références
    Fréquence : 1
    Positions : [399]

Mot : richard
  Document : Références
    Fréquence : 2
    Positions : [414, 716]

Mot : cordal
  Document : Références
    Fréquence : 1
    Positions : [415]

Mot : asia
  Document : Références
    Fréquence : 1
    Positions : [418]

Mot : andamanese
  Document : Références
    Fréquence : 1
    Positions : [420]

Mot : genetic
  Document : Références
    Fréquence : 2
    Positions : [423, 438]

Mot : dispersal
  Document : Références
    Fréquence : 1
    Positions : [429]

Mot : out
  Document : Références
    Fréquence : 1
    Positions : [430]

Mot : africer
  Document : Références
    Fréquence : 1
    Positions : [432]

Mot : american
  Document : Références
    Fréquence : 1
    Positions : [434]

Mot : pmcid
  Document : Références
    Fréquence : 1
    Positions : [444]

Mot : skeletal
  Document : Références
    Fréquence : 1
    


**Laforme générale de l'affichage de l'index inversé enrichi sera :**
(après calcul de IF-IDF
Index inversé enrichi des mots :
Mot : <mot>
  Document : <nom_du_document>
    TF : <fréquence_du_mot_dans_le_document>
    TF-IDF : <valeur_tf_idf>
    Positions : [<liste_des_positions_du_mot_dans_le_document>]

  Document : <nom_du_document_suivant>
    TF : <fréquence_du_mot_dans_le_document_suivant>
    TF-IDF : <valeur_tf_idf>
    Positions : [<liste_des_positions_du_mot_dans_le_document_suivant>]

In [50]:
import math

# fonction pour calculer TF-IDF et enrichir l'index inversé
def calculer_tf_idf(index_inverse, total_documents):
    for mot, documents in index_inverse.items():
        # Calcul de DF pour chaque mot (nombre de documents dans lesquels il apparaît)
        df = len(documents)

        # calcul l'IDF
        idf = math.log(total_documents / df)

        # mettre à jour chaque document pour ce mot avec la pondération TF-IDF
        for doc, info in documents.items():
            # Accées direct à la fréquence TF et aux positions
            tf = info.get('tf', 0)  # Récupèration TF directement depuis la structure correcte
            positions = info.get('positions', [])  # Récupèration positions ou liste vide

            # calcul le TF-IDF
            tf_idf = tf * idf

            # mettre à jour le dictionnaire pour inclure TF, TF-IDF et positions
            index_inverse[mot][doc] = {
                'tf': tf,
                'tf-idf': tf_idf,
                'positions': positions
            }

    return index_inverse

total_documents = len(processed_documents)
index_inverse_enrichi = calculer_tf_idf(index_inverse_enrichi, total_documents)



In [49]:
# fonction pour afficher l'index inversé enrichi
def afficher_index_inverse_enrichi(index_inverse_enrichi):
    print("Index inversé enrichi des mots (avec TF-IDF) :")
    for mot, documents in index_inverse_enrichi.items():
        print(f"\nMot : {mot}")
        for doc, info in documents.items():
            tf = info['tf']
            tf_idf = info['tf-idf']
            positions = info['positions']
            print(f"  Document : {doc}")
            print(f"    TF : {tf}")
            print(f"    TF-IDF : {tf_idf}")
            print(f"    Positions : {positions}")

afficher_index_inverse_enrichi(index_inverse_enrichi)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  Document : Liens externes
    TF : 1
    TF-IDF : 6.405228458030842
    Positions : [9]

Mot : genetiquement
  Document : Liens externes
    TF : 1
    TF-IDF : 6.405228458030842
    Positions : [11]

Mot : viand
  Document : Liens externes
    TF : 1
    TF-IDF : 6.405228458030842
    Positions : [13]

Mot : vitro
  Document : Liens externes
    TF : 1
    TF-IDF : 6.405228458030842
    Positions : [15]

Mot : demodulateur
  Document : Liens externes
    TF : 1
    TF-IDF : 6.405228458030842
    Positions : [1]

Mot : donde
  Document : Liens externes
    TF : 1
    TF-IDF : 6.405228458030842
    Positions : [2]

Mot : gravitationnel
  Document : Liens externes
    TF : 1
    TF-IDF : 6.405228458030842
    Positions : [3]

Mot : ligo
  Document : Liens externes
    TF : 1
    TF-IDF : 6.405228458030842
    Positions : [2]

Mot : virgo
  Document : Liens externes
    TF : 1
    TF-IDF : 6.405228458030842
  